In [1]:
from glob import glob
from openai import OpenAI
from dotenv import load_dotenv
import os
import base64

In [ ]:
load_dotenv()#.env파일에 저장된 환경변수를 불러오는 함수
api_key=os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)#api_key를 이용하여 OpenAI와 통신

def encode_image(image_path):
    with open(image_path, "rb") as image_file: #rb(바이너리모드): 0과1의 이진 데이터 그대로 읽기/ 이미지는 바이트로 구성되어있기때문
        return base64.b64encode(image_file.read()).decode("utf-8")
#이진 데이터를 base64형식(텍스트 형태)으로 인코딩 -> 인코딩된 데이터를 바이트에서 문자열로 변환

In [3]:
#퀴즈 생성 재시도 최대 3번
def image_quiz(image_path, n_trial=0, max_trial=3):
    if n_trial >= max_trial:
        raise Exception ("Failed to generate a quiz,")
    
    base64_image = encode_image(image_path)

    quiz_prompt="""
    제공한 이미지를 바탕으로, 다음과 같은 양식으로 퀴즈를 만들어주세요.
    정답은 1~4 중 하나만 해당하도록 출제하요.
    토익 리스닝 문제 스타일로 문제를 만들어주세요.
    아래는 예시입니다.
    ---예시---

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
    1. 베이커리에서 사람들이 빵을 사는 모습이 담겨 있습니다.
    2. 맨 앞에 서 있는 사람은 빨간색 셔츠를 입었습니다.
    3. 기차를 타기 위해 줄을 서 있는 사람들이 있습니다.
    4. 점원은 노란색 티셔츠를 입었습니다.

    Listening: Which of the following descriptions of the image is incorrect?
    -1. It shows people buying bread at a bakery.
    -2. The person standing at the front is wearing a red shirt.
    -3. There are people lining up to take a train.
    -4. The clerk is wearing a yellow T-shirt.

    정답 : 4. 점원은 노란색 티셔츠가 아닌 파란색 티셔츠를 입었습니다.
    (주의: 정답은 1~4중 하나만 선택하도록 출제하세요.)
    ===
    """

    messages=[
        {
            "role": "user",
            "content":[
                {
                    "type": "text",
                    "text": quiz_prompt
                },
                {
                    "type": "image_url",
                    "image_url":{
                        "url":f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        }
    ]
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
    except Exception as e:
        print("failed\n"+e)
        return image_quiz(image_path, n_trial+1)
    
    content = response.choices[0].message.content

    #content에 Listening문구 포함 되어있으면 -> 정상 퀴즈
    if "Listening:" in content:
        return content, True
    else:
        return image_quiz(image_path, n_trial+1) #실패시 재시도


In [4]:
q = image_quiz("C:/llm01/gpt/busan_dive.jpg")
print(q[0])

    Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
    1. 많은 사람들이 실내 행사에 참석하고 있습니다.
    2. 스크린에 "DIVE 2024 IN BUSAN"이라고 적혀 있습니다.
    3. 모든 참석자들이 노트북을 사용하고 있습니다.
    4. 천장에는 여러 개의 조명이 설치되어 있습니다.

    Listening: Which of the following descriptions of the image is incorrect?
    -1. Many people are attending an indoor event.
    -2. The screen reads "DIVE 2024 IN BUSAN."
    -3. All attendees are using laptops.
    -4. There are several lights installed on the ceiling.

    정답: 3. 모든 참석자들이 노트북을 사용하고 있는 것은 아닙니다.


*다중 문제*

In [5]:
txt=''
no=1
for g in glob('C:/llm01/gpt/images/*.jpg'):
    q, is_suceed = image_quiz(g)

    if not is_suceed:
        continue

    divider = f'## 문제 {no}\n\n'
    print(divider)

    txt += divider
    filename=os.path.basename(g)
    txt += f'![image]({filename})\n\n'

    print(q)
    txt += q + '\n\n----------------\n\n'

    with open('C:/llm01/gpt/images/image_quiz_eng.md','w',encoding='utf-8') as f:
        f.write(txt)

    no+=1

## 문제 1


---

Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?

1. 많은 사람들이 테이블에 앉아 노트북을 사용하고 있습니다.
2. 천장에 많은 조명이 설치되어 있습니다.
3. 스크린에 "DIVE 2024 IN BUSAN"이라는 문구가 적혀 있습니다.
4. 행사장이 커피숍으로 꾸며져 있습니다.

Listening: Which of the following descriptions of the image is incorrect?
-1. Many people are sitting at tables using laptops.
-2. There are many lights installed on the ceiling.
-3. The screen displays the text "DIVE 2024 IN BUSAN."
-4. The venue is set up as a coffee shop.

정답: 4. 행사장은 커피숍으로 꾸며져 있지 않습니다.
## 문제 2


---

Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 노란색 조형물이 보입니다.
2. 건물에 "Local Stitch"라는 글씨가 있습니다.
3. 사람들이 계단을 오르는 모습이 담겨 있습니다.
4. 빨간 벽돌 건물이 보입니다.

Listening: Which of the following descriptions of the image is incorrect?
-1. A yellow sculpture is visible.
-2. The building has the words "Local Stitch" on it.
-3. There are people climbing stairs.
-4. A red brick building is visible.

정답 : 3. 사람들이 계단을 오르는 모습이 보이지 않습니다.

---
## 문제 3


Q: 다음 이미지에 대한 설명 중 옳지 않은 것은 무엇입니까?
1. 사람들이 카운터에서 주문을 하고 있습니다.
2. 노란색 벽이 